Loading the dataset

In [ ]:
import time
import warnings
#warnings.filterwarnings('ignore')
from mnist import MNIST #this is for loading input data
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
emnist = MNIST('/content/mnist_data')
emnist.select_emnist('balanced')

images,labels = emnist.load_training()
testIM,testLAB = emnist.load_testing()

In [ ]:
n_images = np.array(images)
n_labels = np.array(labels)
testIM = np.array(testIM)
testLAB = np.array(testLAB)

visualizing the dataset

In [ ]:
n_images1 = n_images[19000].reshape(28,28)

In [ ]:
plt.imshow(n_images1,cmap='gist_gray')
plt.show()

Normailizing data

In [ ]:
n_images[0][455]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(n_images)
train_images = scaler.transform(n_images)

In [ ]:
train_images[0][455]

In [ ]:
scaler.fit(testIM)
test_images = scaler.transform(testIM)

In [ ]:
testIM[0][567]

In [ ]:
test_images[0][567]

Adding new dataset

In [ ]:
from PIL import ImageFilter,Image
import helper

In [ ]:
count=47
for k in range(1,15):
    for i in range(1,50):
        try:
            im = Image.open('Addeddataset/a{}- ({}).jpg'.format(k,i)).convert('LA')
            print("done")
        except:
            print("error")
            break
        n_image = helper.normalize_to_emnist(im)
        train_images = np.append(train_images,[n_image],axis=0)
        test_images = np.append(test_images,[n_image],axis=0)
        n_labels = np.append(n_labels,[count])
        testLAB = np.append(testLAB,[count])
    count+=1

In [ ]:
print(len(train_images))
print(len(test_images))
print(len(n_labels))
print(len(testLAB))

In [ ]:
%store n_images
%store n_labels
%store testIM 
%store testLAB

In [ ]:
n_labels[113015]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
n_labels[0]

In [ ]:
shaped_n_labels  = n_labels.reshape(-1,1)
enc = OneHotEncoder()
enc.fit(shaped_n_labels)
train_labels = enc.transform(shaped_n_labels).toarray()

In [ ]:
train_labels[0]

In [ ]:
shaped_testLAB = testLAB.reshape(-1,1)
enc.fit(shaped_testLAB)
test_labels = enc.transform(shaped_testLAB).toarray()

In [ ]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)
# initialising bias
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)
#conv2d
def conv2d(x,W):
    #x -> [bias,height,width,channels]
    #W -> [Filter H,filter W,Channel In,Channel Out]
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
#pooling layer
def max_pool_2by2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
#Convolutional layer
def convolutional_layer(input_x,shape):
    W=init_weights(shape)
    bias = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+bias)
#Fully connected layer
def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    bias = init_bias([size])
    return tf.matmul(input_layer,W) + bias

Creating model

In [ ]:
x = tf.placeholder(tf.float32,shape=[None,784])
y_true=tf.placeholder(tf.float32,shape=[None,59])

#layers(input)
x_image = tf.reshape(x,[-1,28,28,1])

#first convolutional layer
convo_1 = convolutional_layer(x_image,shape=[5,5,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

#second convolutional layer
convo_2 = convolutional_layer(convo_1_pooling,shape=[5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

#fully connected layer
convo_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_flat,1024))


optimization

In [ ]:
#drop out (used to overcome overfitting)

hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

y_pred = normal_full_layer(full_one_dropout,59)

#Loss Function 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits = y_pred))

#optimizer
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
train =optimizer.minimize(cross_entropy)

#initialise variables to execute it
init = tf.global_variables_initializer()

steps = 5000

In [ ]:
saver = tf.train.Saver()

Training and saving the model

In [ ]:
start = time.time()
with tf.Session() as sess:
    sess.run(init)
    batch_size = 120
    for i in range(steps):
        rand_ind = np.random.randint(len(train_images),size=batch_size)
        feed = {x:train_images[rand_ind],y_true:train_labels[rand_ind],hold_prob:0.5}
        sess.run(train,feed_dict=feed)
        
        if i%100 == 0:
            print("On step: {}".format(i))
            match = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(match,tf.float32))
            print("Accuracy obtained at {x} is {y}".format(x=i,y=sess.run(acc,feed_dict={x:test_images,y_true:test_labels,hold_prob:1.0})))
            print('\n')
    saver.save(sess,'model/cnn_model_1_with_tamil.ckpt')
end = time.time()

Predicting the input image

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
%store -r n_image

In [ ]:
with tf.Session() as sess:
        sess.run(init)
        saver.restore(sess, "model/cnn_model_1_with_tamil.ckpt")
       
        prediction=tf.argmax(y_pred,1)
        var = prediction.eval(feed_dict={x: [n_image],y_true:train_labels,hold_prob: 0.5}, session=sess)

In [ ]:
labels_dict ={0:0,1:1,2:2,3:3,4:4,5:5,6:6,7:7,8:8,9:9,10:'A',11:'B',12:'C',13:'D',14:'E',15:'F',16:'G',17:'H',18:'I',19:'J',20:'K',21:'l',22:'M',23:'N',24:'O',25:'P',26:'Q',27:'R',28:'S',29:'T',30:'u',31:'V',32:'W',33:'X',34:'Y',35:'Z',36:'a',37:'b',38:'d',39:'e',40:'f',41:'g',42:'h',43:'n',44:'q',45:'r',46:'t',47:'அ',48:'ஆ'}
labels_dict[49]="இ"
labels_dict[50]='ஈ'
labels_dict[51]='உ'
labels_dict[52]='ஊ'
labels_dict[53]='எ'
labels_dict[54]='ஏ'
labels_dict[55]='ஐ'
labels_dict[56]='ஒ'
labels_dict[57]='ஓ'
labels_dict[58]='ஔ'

In [ ]:
print('The predicted character is : "{}"'.format(labels_dict[var[0]]))

In [ ]:
n_images.shape

In [ ]:
%store -r n_images
%store -r n_labels
%store -r testIM 
%store -r testLAB